In [97]:
import requests
import pandas as pd
import datetime as dt
from bs4 import BeautifulSoup
from datetime import datetime
import matplotlib.pyplot as plt

url="http://sarthak.nhmmp.gov.in/covid/facility-bed-occupancy-details/?show=200&pagenum=1&district_id=1&facility_org_type=0&facility=0"

r = requests.get(url, headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
c=r.content
soup=BeautifulSoup(c,"html.parser")

name=soup.find_all("div",{"class":"hospitalname"})

In [98]:
def get_hosp_name(nm):
    start=0
    end=0
    for a in range(len(nm)):
        if nm[a]==">" and start==0:
            start=a+1
        if nm[a]=="/" and end==0:
            end=a
    return str(nm[start:end])

In [99]:
def read_label(nam):
    x=0
    y=0
    for a in range(len(nam)):
        if nam[a]==">" and x==0:
            x=a+1
        if nam[a]=="<" and y==0:
            y=a
    return str(nam[x:y])

In [100]:
def tot_beds(nam):
    x=0
    y=0
    for a in range(1,len(nam)):
        if nam[a]==">" and x==0:
            x=a+1
        if nam[a]=="<" and y==0:
            y=a

    return nam[x:y]

In [101]:
def decode(st):
    x=0
    y=0
    for a in range(len(st)):
        if st[a]=="/":
            x=a-1
            y=a+2
            break

    return[st[:x],st[y:]]

In [102]:
def decode_date(st):
    start=0
    end=0
    for i in range(len(st)):
        try:
            int(st[i])
            start=i
            break
        except:
            pass
    for i in range(len(st),0,-1):
        try:
            int(st[i])
            end=i+1
            break
        except:
            pass
        
    return st[start:end]

In [103]:
hospital_list=[]
for a in range(len(name)):
    hospital_list.append(get_hosp_name(str(name[a])))
for a in range(len(hospital_list)):
    try:
        hospital_list[a]=hospital_list[a].replace("&amp;","")
    except:
        pass

In [104]:
beds=soup.find_all("td",{"class":"text-center"})

isolation=[]
oxygen=[]
icu=[]
for a in range(len(beds)):
    labels=beds[a].find_all("label")
    for b in range(3):
        labels[b]=str(labels[b])
    icu.append(int(read_label(labels[2])))
    oxygen.append(int(read_label(labels[1])))
    isolation.append(int(read_label(labels[0])))

In [105]:
total=soup.find_all("span",{"class":"badge bed-status"})
total_beds=[]
remaining_beds=[]
ret=[]

for a in range(len(total)):
    l=decode(tot_beds(str(total[a])))
    total_beds.append(int(l[0]))
    remaining_beds.append(int(l[1]))

In [106]:
total=soup.find_all("div",{"class":"last-updated"})
last_updated=[]
for a in range(len(total)):
    st=decode_date(str(total[a]))
    st=st.replace(",","")
    date_time_str = st
    date_time_obj = dt.datetime.strptime(date_time_str, '%d-%m-%Y %H:%M:%S')
    last_updated.append(date_time_obj)

In [107]:
df = pd.DataFrame([hospital_list,total_beds,remaining_beds,icu,oxygen,isolation,last_updated])
df=df.T
df.columns =['Hospital','Total','Remaining','ICU','Oxygen','Isolation','Last Updated']

In [108]:
icu_a=len(df[df["ICU"]>0])
oxy_a=len(df[df["Oxygen"]>0])
iso_a=len(df[df["Isolation"]>0])
any_bed=len(df[(df["ICU"]>0) | (df["Isolation"]>0) | (df["Oxygen"]>0)])
print(icu_a,oxy_a,iso_a,any_bed)

14 17 32 44


In [109]:
icu_hosp=(df[df["ICU"]>0])
oxy_hosp=(df[df["Oxygen"]>0])
iso_hosp=(df[df["Isolation"]>0])
any_bed_hosp=df[(df["ICU"]>0) | (df["Isolation"]>0) | (df["Oxygen"]>0)]
oi_hosp=df[(df["ICU"]>0) |  (df["Oxygen"]>0)]

In [110]:
a=df.sort_values(by="Total",ascending=False,ignore_index=True)

In [111]:
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y")

In [112]:
#analysis generator



In [113]:
file="analysis/EOD "+dt_string+".xlsx"
filename = pd.ExcelWriter(file, engine='xlsxwriter')
a.to_excel(filename,sheet_name="All Hospitals",index=False)
icu_hosp.to_excel(filename,sheet_name="ICU Availabe",index=False)
oxy_hosp.to_excel(filename,sheet_name="Oxygen Available",index=False)
iso_hosp.to_excel(filename,sheet_name="Isolation Available",index=False)
oi_hosp.to_excel(filename,sheet_name="ICU or Oxygen Available",index=False)
any_bed_hosp.to_excel(filename,sheet_name="Any Bed Available",index=False)
filename.save()